In [ ]:
# Definindo variáveis iniciais
import estrutura_ud
import os

root_deprel = ["root"]
subordinate_deprel = ["acl:relcl", "ccomp", "advcl"]
max_examples = 20

path = {
    'bosque': "bosque-ud-2.6.conllu",
    'dhbb': "dhbb.conllu",
    "obras": "obras.conllu"
}

In [ ]:
# Carregamento dos corpora (demora muito!)
bosque = estrutura_ud.Corpus(recursivo=True)
dhbb = estrutura_ud.Corpus(recursivo=True)
obras = estrutura_ud.Corpus(recursivo=True)
bosque.load(path["bosque"])
dhbb.load(path["dhbb"])
obras.load(path["obras"])

corpora = {
    'dhbb': dhbb,
    'obras': obras,
    'bosque': bosque
}

In [4]:
# Estatísticas dos corpora
stats_corpora = {}
for corpus in corpora:
    stats_corpora[corpus] = {
        "size": os.path.getsize(path[corpus])/1024, 
        "n_tokens": 0, 
        "n_sentences": 0,
    }
    
for corpus in corpora:
    for sentence in corpora[corpus].sentences.values():
        stats_corpora[corpus]['n_sentences'] += 1
        for token in sentence.tokens:
            if not '-' in token.id:
                stats_corpora[corpus]['n_tokens'] += 1
    
print(stats_corpora)

{'dhbb': {'size': 960964.294921875, 'n_tokens': 16037286, 'n_sentences': 480218}, 'obras': {'size': 480188.513671875, 'n_tokens': 7863261, 'n_sentences': 353662}, 'bosque': {'size': 14846.041015625, 'n_tokens': 227825, 'n_sentences': 9364}}


In [5]:
# Números de orações
total_deprel = {}
for corpus in corpora:
    total_deprel[corpus] = {
        "|".join(root_deprel): 0,
        "|".join(subordinate_deprel): 0
    }
    for sentence in corpora[corpus].sentences.values():
        for token in sentence.tokens:
            if token.deprel in root_deprel:
                total_deprel[corpus]["|".join(root_deprel)] += 1
            if token.deprel in subordinate_deprel:
                total_deprel[corpus]["|".join(subordinate_deprel)] += 1
print(total_deprel)

{'dhbb': {'root': 480218, 'acl:relcl|ccomp|advcl': 341133}, 'obras': {'root': 353662, 'acl:relcl|ccomp|advcl': 316297}, 'bosque': {'root': 9364, 'acl:relcl|ccomp|advcl': 6842}}


# Busca ingênua tanto em root quanto em subordinada

In [6]:
def find_clause_without_subject(sentences, deprel_list):
    clauses_without_subject = {}
    for sent_id, sentence in sentences.items():
        for t, token in enumerate(sentence.tokens):
            if token.deprel in deprel_list:
                has_subj = False
                for _token in sentence.tokens:
                    if _token.dephead == token.id and _token.deprel in ["nsubj", "csubj", "nsubj:pass"]:
                        has_subj = True
                        break
                if not has_subj:
                    if not sent_id in clauses_without_subject:
                        clauses_without_subject[sent_id] = []
                    clauses_without_subject[sent_id].append(t)
    return clauses_without_subject

root_without_subject = {}
subordinate_without_subject = {}
for corpus in corpora:
    root_without_subject[corpus] = find_clause_without_subject(corpora[corpus].sentences, root_deprel)
    subordinate_without_subject[corpus] = find_clause_without_subject(corpora[corpus].sentences, subordinate_deprel)

In [7]:
for corpus in corpora:
    print("\n{} - orações principais com sujeito oculto ingênuo: {} / {}".format(corpus, sum([len(x) for x in root_without_subject[corpus].values()]), total_deprel[corpus]["|".join(root_deprel)]))
    print("{} - orações subordinadas com sujeito oculto ingênuo: {} / {}".format(corpus, sum([len(x) for x in subordinate_without_subject[corpus].values()]), total_deprel[corpus]["|".join(subordinate_deprel)]))

for corpus in corpora:
    i = 0
    ii = 0
    print("\n=== {} exemplos de sujeito oculto na oração principal em {} ===".format(max_examples, corpus))
    for sent_id, tokens in root_without_subject[corpus].items():
        i += len(tokens)
        print(sent_id + ": " + " ".join([x.word if t not in tokens else "*" + x.word + "*" for t, x in enumerate(corpora[corpus].sentences[sent_id].tokens) if not '-' in x.id]))
        if i >= max_examples:
            break
    print("\n=== {} exemplos de sujeito oculto na oração subordinada em {} ===".format(max_examples, corpus))
    for sent_id, tokens in subordinate_without_subject[corpus].items():
        ii += len(tokens)
        print(sent_id + ": " + " ".join([x.word if t not in tokens else "*" + x.word + "*" for t, x in enumerate(corpora[corpus].sentences[sent_id].tokens) if not '-' in x.id]))
        if ii >= max_examples:
            break


=== 20 exemplos de sujeito oculto na oração principal em dhbb ===
2: *Estudou* em o Ginásio Diocesano de São Paulo e bacharelou se em 1910 por a Faculdade de Ciências Jurídicas e Sociais .
3: Dedicando se a a advocacia , foi *promotor* público em Cunha ( SP ) e depois delegado de polícia em o Rio de Janeiro , então Distrito Federal .
4: *Iniciou* sua vida política como deputado federal por o Distrito Federal , exercendo o mandato de 1927 a 1929 .
6: Ligado a o governo federal , *encontrava* se a o lado de o presidente Washington Luís , em o palácio Guanabara , em o momento de sua deposição em o dia 24 de outubro .
7: Junto com outros companheiros também solidários a o regime deposto e que se haviam asilado em embaixadas e legações , foi *enviado* em novembro para o estrangeiro .
8: Em outubro de 1932 , estava *presente* em o porto de Alcântara , em Lisboa , para receber os revolucionários constitucionalistas exilados por o governo de Getúlio Vargas após a derrota de a revolução irromp

CF896-2: De os fumantes , 18,1 milhões são *homens* e 12,5 milhões , mulheres .
CF897-2: De a *FT*
CF899-2: Em a sua administração , *executou* um plano viário com largas avenidas interligando os bairros , que provocaram um boom imobiliário .
CF902-1: *Pedro* Ivo , 63 , tel. 289-3609 )
CF902-3: Quem não quiser beber durante o jantar , *poderá* levar o champanhe para casa .
CF902-4: *Preço* : 18 URVs ( por pessoa )
CF902-5: *Horário* : em o dia 12 , de as 19h a o último cliente
CF902-6: *Cartões* : aceita todos
CF902-7: *Estacionamento* com manobrista
CF903-6: *Folha* -- o que a mulher precisa hoje , em termos de roupas ?
CF904-6: É *o* que confirmam as estatísticas de o Campeonato Brasileiro , encerrado ontem com o novo título palmeirense .
CF908-4: Diariamente , está *promovendo* desfiles de moda para seus consumidores .
CF910-2: Não *houve* acordo para uma trégua durante a Copa .
CF911-1: Era uma *época* em que os cariocas imperavam em campo e fora de ele .
CF911-6: Já não *há* o imp

# Filtro A

In [8]:
def find_haver_impessoal(sentences, list_tokens):
    haver_impessoal = {}
    for sent_id in list_tokens:
        sentence = sentences[sent_id]
        for t, token in enumerate(sentence.tokens):
            if t in list_tokens[sent_id] and token.lemma == "haver" and "Number=Sing" in token.feats and "Person=3" in token.feats:
                if not sent_id in haver_impessoal:
                    haver_impessoal[sent_id] = []
                haver_impessoal[sent_id].append(t)
    return haver_impessoal
                
root_haver_impessoal = {}
subordinate_haver_impessoal = {}
for corpus in corpora:
    root_haver_impessoal[corpus] = find_haver_impessoal(corpora[corpus].sentences, root_without_subject[corpus])
    subordinate_haver_impessoal[corpus] = find_haver_impessoal(corpora[corpus].sentences, subordinate_without_subject[corpus])
    
    for sentence in root_haver_impessoal[corpus]:
        for token in root_haver_impessoal[corpus][sentence]:
            root_without_subject[corpus][sentence].remove(token)
    for sentence in subordinate_haver_impessoal[corpus]:
        for token in subordinate_haver_impessoal[corpus][sentence]:
            subordinate_without_subject[corpus][sentence].remove(token)

In [9]:
for corpus in corpora:
    print("\n{} - orações principais com sujeito oculto e haver impessoal: {}".format(corpus, sum([len(x) for x in root_haver_impessoal[corpus].values()])))
    print("{} - orações subordinadas com sujeito oculto e haver impessoal: {}".format(corpus, sum([len(x) for x in subordinate_haver_impessoal[corpus].values()])))

for corpus in corpora:
    i = 0
    ii = 0
    print("\n=== {} exemplos de haver impessoal na oração principal em {} ===".format(max_examples, corpus))
    for sent_id, tokens in root_haver_impessoal[corpus].items():
        i += len(tokens)
        print(sent_id + ": " + " ".join([x.word if t not in tokens else "*" + x.word + "*" for t, x in enumerate(corpora[corpus].sentences[sent_id].tokens) if not '-' in x.id]))
        if i >= max_examples:
            break
    print("\n=== {} exemplos de haver impessoal na oração subordinada em {} ===".format(max_examples, corpus))
    for sent_id, tokens in subordinate_haver_impessoal[corpus].items():
        ii += len(tokens)
        print(sent_id + ": " + " ".join([x.word if t not in tokens else "*" + x.word + "*" for t, x in enumerate(corpora[corpus].sentences[sent_id].tokens) if not '-' in x.id]))
        if ii >= max_examples:
            break


=== 20 exemplos de haver impessoal na oração principal em dhbb ===
486: *Há* entretanto quem julgue , como Alexandre Barbosa Lima Sobrinho , que suas realizações foram motivadas por o afã de projetar nacionalmente o próprio nome de Antônio Carlos , tendo em vista a sucessão de Washington Luís , que assumira a presidência de a República em 15 de novembro de 1926 .
553: *Houve* idas e vindas a propósito de a data em que seria possível deflagrar a luta armada .
639: *Houve* críticas , formuladas , segundo Helena Bomeny , « a partir de o pressuposto de que o pacto entre a legião e a Igreja iria favorecer o desenvolvimento de a nova agremiação por a propaganda que a última faria em seu favor .
693: *Há* uma relação de contraponto entre massa e César .
1089: Já *havia* , de esse modo , cumprido a pena de um ano e quatro meses de reclusão a que viria a ser condenado em 1928 , devido a a sua participação em a revolta .
2403: Em o PRP , *houve* desde logo forte resistência a apoiá lo .
2483: N

8390: Diante de a sentença , alegou que não *havia* nenhuma irregularidade .
9072: Em maio de 2008 , por seis votos a cinco , incluindo seu voto favorável , a ação foi julgada improcedente , permitindo se o uso , em pesquisas científicas , de as células-tronco , desde que inviáveis para implantação em o útero materno , congeladas *há* mais de três anos e mediante autorização de os genitores .

=== 20 exemplos de haver impessoal na oração principal em obras ===
33: *Havia* vinte e tantos anos que em torno de ele se formara , de dia para dia , a mais sólida reputação de virtude e santidade .
44: Em uma de elas *havia* um grande nicho com a imagem de a Virgem de a Conceição , quase de tamanho natural ; a um de os cantos , uma negra estante , toscamente feita , pejada de grossos alfarrábios amarelecidos por o tempo ; em o centro , uma mesa de madeira escura com um breviário em cima , a o lado de uma candeia de azeite , um pedaço de pão duro e um cilício cru ; junto a a mesa , um banco de p

CF833-1: Parágrafo 3º. -- Em os contratos celebrados ou convertidos em URV , em que *haja* cláusula de reajuste de valor por índice de preços ou por índice que reflita a variação ponderada de os custos de os insumos utilizados , o cálculo de esses índices , para efeitos de reajuste , deverá ser em esta moeda até a emissão de o real e , de aí em diante , em real , observado o art. 38 de a lei nº. 8.880 de 27.05.94 .
CF845-2: Vamos verificar o que *houve* com essas matérias-primas . »
CF971-6: E como não *há* crescimento sem educação , a causa de as causas para nós é a elevação de o nível educacional de o povo brasileiro .
CP5-9: « A proposta é muito bem vista , porque será mais vantajosa de o que se *houver* só um pólo de interesse em o local » , afirmou o major Carlos Barbosa , de as relações públicas de a Força Aérea , admitindo que , com o parque temático , « se o interesse for diversificado , toda a gente fica a ganhar » .
CP9-1: Co-produção franco-egípcia , « O Emigrante » inspira 

# Filtro B

In [10]:
def find_nominals(sentences, list_tokens):
    nominals = {}
    for sent_id in list_tokens:
        sentence = sentences[sent_id]
        for t, token in enumerate(sentence.tokens):
            if t in list_tokens[sent_id] and token.upos != "VERB":
                is_cop = False
                for _token in sentence.tokens:
                    if _token.head_token.id == token.id and _token.deprel == "cop":
                        is_cop = True
                        break
                if not is_cop:
                    if not sent_id in nominals:
                        nominals[sent_id] = []
                    nominals[sent_id].append(t)
    return nominals
                
root_nominals = {}
subordinate_nominals = {}
for corpus in corpora:
    root_nominals[corpus] = find_nominals(corpora[corpus].sentences, root_without_subject[corpus])
    subordinate_nominals[corpus] = find_nominals(corpora[corpus].sentences, subordinate_without_subject[corpus])
    
    for sentence in root_nominals[corpus]:
        for token in root_nominals[corpus][sentence]:
            root_without_subject[corpus][sentence].remove(token)
    for sentence in subordinate_nominals[corpus]:
        for token in subordinate_nominals[corpus][sentence]:
            subordinate_without_subject[corpus][sentence].remove(token)

In [11]:
for corpus in corpora:
    print("\n{} - orações principais com sujeito oculto e nominal: {}".format(corpus, sum([len(x) for x in root_nominals[corpus].values()])))
    print("{} - orações subordinadas com sujeito oculto e nominal: {}".format(corpus, sum([len(x) for x in subordinate_nominals[corpus].values()])))

for corpus in corpora:
    i = 0
    ii = 0
    print("\n=== {} exemplos de nominais como oração principal em {} ===".format(max_examples, corpus))
    for sent_id, tokens in root_nominals[corpus].items():
        i += len(tokens)
        print(sent_id + ": " + " ".join([x.word if t not in tokens else "*" + x.word + "*" for t, x in enumerate(corpora[corpus].sentences[sent_id].tokens) if not '-' in x.id]))
        if i >= max_examples:
            break
    print("\n=== {} exemplos de nominais como oração subordinada em {} ===".format(max_examples, corpus))
    for sent_id, tokens in subordinate_nominals[corpus].items():
        ii += len(tokens)
        print(sent_id + ": " + " ".join([x.word if t not in tokens else "*" + x.word + "*" for t, x in enumerate(corpora[corpus].sentences[sent_id].tokens) if not '-' in x.id]))
        if ii >= max_examples:
            break


=== 20 exemplos de nominais como oração principal em dhbb ===
28: *Deputados* em fevereiro seguinte .
110: As *Oposições*
117: *Magalhães* , o voto de os representantes classistas e de os republicanos paulistas e ainda o beneplácito de Valadares .
120: *Transcorreria* em 1937 a campanha para a sucessão presidencial .
133: *Pinto* , visando promover uma manifestação de personalidades que contestavam o Estado Novo .
138: « *Manifesto* de os mineiros » , primeiro pronunciamento público de setores liberais contra o Estado Novo .
148: Aleixo fora *um* de os que propuseram o nome de Eduardo Gomes para as eleições presidenciais de 2 de dezembro de 1945 .
150: Dutra , *ministro* de a Guerra de Vargas , tendo como defensores Benedito Valadares e vários outros políticos ligados a o Estado Novo .
157: *Dutra* , que contava com o apoio de o PTB .
187: Em meados de 1963 as articulações políticas entre civis e militares que visavam a a derrubada de Goulart já estavam bem desenvolvidas , contando pr

98: *Deus* a os desamparados por os homens .
103: Eis a o que nem o próprio Ozéas seria capaz de responder *.*
107: Um *enjeitado* !
109: Mas um *enjeitado* de quem ? ...
116: *Sim* !
117: *Sim* !

=== 20 exemplos de nominais como oração subordinada em obras ===
139: Mas tal era a confiança que o *velho*
188: « O amado de o meu coração é bonito que nem essa *cabra* arisca , que grimpa a a tardinha por os escalvados outeiros sem relva , e que de noite e de manhã a gente não bispa mais .
200: São leves , macios , correntes e ondulosos , são como os cabelos de o milho doce , e mais doce que o *mel* gostoso de a flor de a banana .
237: As macerações de os jejuns e de as ásperas disciplinas não conseguiram desfibrar lhe de todo a sólida compleição com que a *natureza* o dotara .
305: Há por aqui milhares de teólogos , muito mais fortes em a matéria e mais oradores de o que Ângelo , que não conseguem abalar um só de os seus ouvintes ! -- Então o que é que *foi* ? ...
395: -- Sim , tornou Cob

# Filtro C

In [12]:
fenomenos_natureza = "chover|ventar|anoitecer|amanhecer|entardecer|relampejar|trovejar|escurecer|clarear".split("|")
def find_natureza(sentences, list_tokens):
    natureza = {}
    for sent_id in list_tokens:
        sentence = sentences[sent_id]
        for t, token in enumerate(sentence.tokens):
            if t in list_tokens[sent_id] and token.lemma in fenomenos_natureza:
                if not sent_id in natureza:
                    natureza[sent_id] = []
                natureza[sent_id].append(t)
    return natureza
                
root_natureza = {}
subordinate_natureza = {}
for corpus in corpora:
    root_natureza[corpus] = find_natureza(corpora[corpus].sentences, root_without_subject[corpus])
    subordinate_natureza[corpus] = find_natureza(corpora[corpus].sentences, subordinate_without_subject[corpus])
    
    for sentence in root_natureza[corpus]:
        for token in root_natureza[corpus][sentence]:
            root_without_subject[corpus][sentence].remove(token)
    for sentence in subordinate_natureza[corpus]:
        for token in subordinate_natureza[corpus][sentence]:
            subordinate_without_subject[corpus][sentence].remove(token)

In [13]:
for corpus in corpora:
    print("\n{} - orações principais com sujeito oculto e fenômeno natureza: {}".format(corpus, sum([len(x) for x in root_natureza[corpus].values()])))
    print("{} - orações subordinadas com sujeito oculto e fenômeno natureza: {}".format(corpus, sum([len(x) for x in subordinate_natureza[corpus].values()])))
    
for corpus in corpora:
    i = 0
    ii = 0
    print("\n=== {} exemplos de fenômeno natureza como oração principal em {} ===".format(max_examples, corpus))
    for sent_id, tokens in root_natureza[corpus].items():
        i += len(tokens)
        print(sent_id + ": " + " ".join([x.word if t not in tokens else "*" + x.word + "*" for t, x in enumerate(corpora[corpus].sentences[sent_id].tokens) if not '-' in x.id]))
        if i >= max_examples:
            break
    print("\n=== {} exemplos de fenômeno natureza como oração subordinada em {} ===".format(max_examples, corpus))
    for sent_id, tokens in subordinate_natureza[corpus].items():
        ii += len(tokens)
        print(sent_id + ": " + " ".join([x.word if t not in tokens else "*" + x.word + "*" for t, x in enumerate(corpora[corpus].sentences[sent_id].tokens) if not '-' in x.id]))
        if ii >= max_examples:
            break


=== 20 exemplos de fenômeno natureza como oração principal em dhbb ===

=== 20 exemplos de fenômeno natureza como oração subordinada em dhbb ===
5692: Que o Lula , de botar tanto poste sem luz , acabe *escurecendo* o Brasil .
82280: Em Cachoeira de o Sul , o levante se desenrolou sem qualquer contratempo : em o *entardecer* de o dia 4 de outubro , as unidades federais de a cidade já se achavam reorganizadas por os revolucionários , prontas para seguir para o norte de o estado , de onde rumariam para São Paulo e Rio .
337330: Em o segundo dia de luta , uma companhia de o 5º RI e todo o 6º RI , comandado por o major Raul Cabral Velho , aderiram a a revolta , mas a o *anoitecer* os primeiros reforços legalistas começaram a penetrar em a cidade .
408100: Em Cachoeira de o Sul , o levante se desenrolou sem qualquer contratempo : em o *entardecer* de o dia 4 de outubro , as unidades federais de a cidade já se achavam reorganizadas por os revolucionários , prontas para seguir para o norte de

# Filtro D

In [14]:
def find_se(sentences, list_tokens):
    se = {"a": {}, "b": {}, "c": {}}
    for sent_id in list_tokens:
        sentence = sentences[sent_id]
        for t, token in enumerate(sentence.tokens):
            if token.head_token.id != "_" and token.lemma == "se" and token.upos == "PRON" and "Gender=Unsp" in token.feats and sentence.map_token_id[token.head_token.id] in list_tokens[sent_id]:
                if not sent_id in se["a"]:
                    se["a"][sent_id] = []
                se["a"][sent_id].append(sentence.map_token_id[token.head_token.id])
            if token.head_token.id != "_" and token.lemma == "se" and token.upos == "PRON" and "VerbForm=Inf" in token.head_token.feats and sentence.map_token_id[token.head_token.id] in list_tokens[sent_id]:
                if not sent_id in se["b"]:
                    se["b"][sent_id] = []
                se["b"][sent_id].append(sentence.map_token_id[token.head_token.id])
            if token.head_token.id != "_" and token.lemma == "se" and token.upos == "PRON" and sentence.map_token_id[token.head_token.id] in list_tokens[sent_id]:
                for _token in sentence.tokens:
                    if _token.deprel == "case" and _token.head_token.head_token.id == token.head_token.id:
                        if not sent_id in se["c"]:
                            se["c"][sent_id] = []
                        se["c"][sent_id].append(sentence.map_token_id[token.head_token.id])
                        break
    return se
                
root_se = {}
subordinate_se = {}
for corpus in corpora:
    root_se[corpus] = find_se(corpora[corpus].sentences, root_without_subject[corpus])
    subordinate_se[corpus] = find_se(corpora[corpus].sentences, subordinate_without_subject[corpus])
    
    for sentence in root_se[corpus]["a"]:
        for token in root_se[corpus]["a"][sentence]:
            if token in root_without_subject[corpus][sentence]:
                root_without_subject[corpus][sentence].remove(token)
    for sentence in subordinate_se[corpus]["a"]:
        for token in subordinate_se[corpus]["a"][sentence]:
            if token in subordinate_without_subject[corpus][sentence]:
                subordinate_without_subject[corpus][sentence].remove(token)
            
    for sentence in root_se[corpus]["b"]:
        for token in root_se[corpus]["b"][sentence]:
            if token in root_without_subject[corpus][sentence]:
                root_without_subject[corpus][sentence].remove(token)
    for sentence in subordinate_se[corpus]["b"]:
        for token in subordinate_se[corpus]["b"][sentence]:
            if token in subordinate_without_subject[corpus][sentence]:
                subordinate_without_subject[corpus][sentence].remove(token)
            
    for sentence in root_se[corpus]["c"]:
        for token in root_se[corpus]["c"][sentence]:
            if token in root_without_subject[corpus][sentence]:
                root_without_subject[corpus][sentence].remove(token)
    for sentence in subordinate_se[corpus]["c"]:
        for token in subordinate_se[corpus]["c"][sentence]:
            if token in subordinate_without_subject[corpus][sentence]:
                subordinate_without_subject[corpus][sentence].remove(token)

In [15]:
for corpus in corpora:
    for filtro in ["a", "b", "c"]:
        print("\n{} - orações principais com sujeito oculto e se (filtro {}): {}".format(corpus, filtro, sum([len(x) for x in root_se[corpus][filtro].values()])))
        print("{} - orações subordinadas com sujeito oculto e se (filtro {}): {}".format(corpus, filtro, sum([len(x) for x in subordinate_se[corpus][filtro].values()])))

for corpus in corpora:
    for filtro in ["a", "b", "c"]:
        i = 0
        ii = 0
        print("\n=== {} exemplos de se (filtro {}) como filho de oração principal em {} ===".format(max_examples, filtro, corpus))
        for sent_id, tokens in root_se[corpus][filtro].items():
            i += len(tokens)
            print(sent_id + ": " + " ".join([x.word if t not in tokens else "*" + x.word + "*" for t, x in enumerate(corpora[corpus].sentences[sent_id].tokens) if not '-' in x.id]))
            if i >= max_examples:
                break
        print("\n=== {} exemplos de se (filtro {}) como filho de oração subordinada em {} ===".format(max_examples, filtro, corpus))
        for sent_id, tokens in subordinate_se[corpus][filtro].items():
            ii += len(tokens)
            print(sent_id + ": " + " ".join([x.word if t not in tokens else "*" + x.word + "*" for t, x in enumerate(corpora[corpus].sentences[sent_id].tokens) if not '-' in x.id]))
            if ii >= max_examples:
                break


=== 20 exemplos de se (filtro a) como filho de oração principal em dhbb ===
4288: Em o início de 1966 , a o se *formalizar* a criação de dois novos partidos políticos , a Aliança Renovadora Nacional ( Arena ) , de apoio a o governo , e o Movimento Democrático Brasileiro ( MDB ) , de oposição , filiou se a a Arena , tendo sido o primeiro presidente de esse partido em a Guanabara .
13465: *Recomendava* ainda que se de esse « preferência a o regime estatal » para a exploração de a indústria siderúrgica , propondo a criação de o Conselho Nacional de Minas e Metalurgia e de o Conselho Nacional de Carvão Mineral , além de a nacionalização de os bancos de depósito .
26479: *Transferindo* se para a Inglaterra e obteve o título de Mestrado e Doutorado em a Hertford College em a Universidade de Oxford , em Oxford .
34231: *Veja* se que « absoluta prioridade » não é simplesmente uma expressão , mas um princípio constitucional que gera direitos e obrigações jurídicas .
51684: *Mudando* se para Po

45542: Em 1973 , devido a o afastamento de Quércia de a prefeitura para *candidatar* se a o
46831: A o se *aproximarem* as eleições de outubro de 1950 , considerando que a situação havia mudado -- e que , mais uma vez , Eduardo Gomes não teria condições de se eleger presidente -- Chateaubriand deu um discreto apoio a a candidatura de Vargas .
47152: Segundo declarou a a época a a imprensa , iria se *dedicar* a pavimentar seu caminho para a presidência de a República , a a qual , como presidente de honra de o PFL , se considerava candidato nato . Íris
56343: Em agosto de 1935 , a o se *desligar* de a ANL , Miguel Costa escreveu uma carta a o líder comunista defendendo o programa de a entidade , mas afirmando : « Você , naturalmente pouco informado , supondo que o movimento de a ANL tivesse tanto de profundidade como de extensão , lançou seu manifesto , dando a sua palavra de ordem de todo o poder a a ANL ' , brado profundamente revolucionário , subversivo , aconselhável a os momentos qu

293: Estudante universitário , *filiou* se a o Partido de o Movimento Democrático Brasileiro ( PMDB ) .
299: Em 1990 , *filiou* se a o Partido Socialista Brasileiro ( PSB ) , por o qual concorreu a as eleições para a Assembleia Legislativa de Pernambuco .
328: *Empenhou* se também em a aprovação de a Lei de Biossegurança , sancionada por o presidente Lula em o dia 24 e março de 2005 , que regulamentou a produção de transgênicos em o país e autorizou a pesquisa científica com células-tronco .
351: *Candidatou* se a a presidência de o Brasil em chapa constituída com Marina Silva , recém-filiada a o PSB , candidata a vice .
359: *Casou* se com Renata de Andrade Lima
370: Com o fim de o Estado Novo ( 1937-1945 ) e a redemocratização de o país , *elegeu* se em o pleito de dezembro de 1945 deputado a a Assembléia Nacional Constituinte por o Pará em a legenda de a União Democrática Nacional ( UDN ) , assumindo o mandato em fevereiro de o ano seguinte .
383: Com a extinção de os partidos polít

7363: *Achavam* se em o mesmo ponto de abandono , viram se companheiros de solidão , e é natural que se compreendessem e que se tornassem afinal amigos inseparáveis .
11311: Mas , ah ! justamente quando esta vai fugindo , é que a mulher mais a exige em os seus amantes ; a a moça , bonita e cheia de vida , pouco importa que o homem a quem se *dá* seja tão novo e tão lindo como ela ; para o seu completo deleite chegam lhe os próprios encantos e , vaidosa , contenta se com ser admirada e não precisa admirar ninguém .
12167: O criado , surpreso com estas palavras , mas sem o dar a perceber , afastou se imediatamente ; a o passo que o amo , *vestindo* se a as pressas e , contra o seu costume , em desalinho , abandonou ainda uma vez o gabinete e ganhou em direitura a o quarto de o enfermo .
14621: E , porque a ama fizesse um certo ar de estranheza : -- Para não manchar o colchão , como não ? -- Ah ... fez Magdá , *caindo* em si . --
14875: O populacho de o cortiço e os trabalhadores de a ped

dhbb - orações subordinadas com sujeito oculto e se (filtro a): 757

dhbb - orações principais com sujeito oculto e se (filtro b): 79
dhbb - orações subordinadas com sujeito oculto e se (filtro b): 2692

dhbb - orações principais com sujeito oculto e se (filtro c): 26425
dhbb - orações subordinadas com sujeito oculto e se (filtro c): 13189

obras - orações principais com sujeito oculto e se (filtro a): 215
obras - orações subordinadas com sujeito oculto e se (filtro a): 154

obras - orações principais com sujeito oculto e se (filtro b): 229
obras - orações subordinadas com sujeito oculto e se (filtro b): 1609

obras - orações principais com sujeito oculto e se (filtro c): 4915
obras - orações subordinadas com sujeito oculto e se (filtro c): 6447

bosque - orações principais com sujeito oculto e se (filtro a): 31
bosque - orações subordinadas com sujeito oculto e se (filtro a): 5

bosque - orações principais com sujeito oculto e se (filtro b): 2
bosque - orações subordinadas com sujeito

# Contagem final

In [16]:
for corpus in corpora:
    print("\n{} - orações principais com sujeito oculto final: {} / {}".format(corpus, sum([len(x) for x in root_without_subject[corpus].values()]), total_deprel[corpus]["|".join(root_deprel)]))
    print("{} - orações subordinadas com sujeito oculto final: {} / {}".format(corpus, sum([len(x) for x in subordinate_without_subject[corpus].values()]), total_deprel[corpus]["|".join(subordinate_deprel)]))

for corpus in corpora:
    i = 0
    ii = 0
    print("\n=== {} exemplos de sujeito oculto na oração principal em {} ===".format(max_examples, corpus))
    for sent_id, tokens in root_without_subject[corpus].items():
        if len(tokens):
            i += len(tokens)
            print(sent_id + ": " + " ".join([x.word if t not in tokens else "*" + x.word + "*" for t, x in enumerate(corpora[corpus].sentences[sent_id].tokens) if not '-' in x.id]))
            if i >= max_examples:
                break
    print("\n=== {} exemplos de sujeito oculto na oração subordinada em {} ===".format(max_examples, corpus))
    for sent_id, tokens in subordinate_without_subject[corpus].items():
        if len(tokens):
            ii += len(tokens)
            print(sent_id + ": " + " ".join([x.word if t not in tokens else "*" + x.word + "*" for t, x in enumerate(corpora[corpus].sentences[sent_id].tokens) if not '-' in x.id]))
            if ii >= max_examples:
                break


=== 20 exemplos de sujeito oculto na oração principal em dhbb ===
2: *Estudou* em o Ginásio Diocesano de São Paulo e bacharelou se em 1910 por a Faculdade de Ciências Jurídicas e Sociais .
3: Dedicando se a a advocacia , foi *promotor* público em Cunha ( SP ) e depois delegado de polícia em o Rio de Janeiro , então Distrito Federal .
4: *Iniciou* sua vida política como deputado federal por o Distrito Federal , exercendo o mandato de 1927 a 1929 .
7: Junto com outros companheiros também solidários a o regime deposto e que se haviam asilado em embaixadas e legações , foi *enviado* em novembro para o estrangeiro .
8: Em outubro de 1932 , estava *presente* em o porto de Alcântara , em Lisboa , para receber os revolucionários constitucionalistas exilados por o governo de Getúlio Vargas após a derrota de a revolução irrompida em julho de esse ano em São Paulo .
10: *Obteve* uma suplência e , em julho de 1946 , foi convocado para participar de os trabalhos constituintes .
11: Com a promulgaç

27: O arcebispo , já desesperado , ia estender o braço para *tomar* a o acaso o primeiro de os seus sufragâneos , e ordenar lhe que *subisse* a o púlpito e despejasse , com um milhão de raios ! um sermão qualquer , quando de improviso rasgou se o reposteiro de a sala , em que ele se achava entre uma negra nuvem de batinas , e viu se surgir a veneranda figura de frei Ozéas , com as suas grandes barbas brancas e a sua enorme calva de profeta .

=== 20 exemplos de sujeito oculto na oração principal em bosque ===
CF896-2: De os fumantes , 18,1 milhões são *homens* e 12,5 milhões , mulheres .
CF899-2: Em a sua administração , *executou* um plano viário com largas avenidas interligando os bairros , que provocaram um boom imobiliário .
CF902-3: Quem não quiser beber durante o jantar , *poderá* levar o champanhe para casa .
CF904-6: É *o* que confirmam as estatísticas de o Campeonato Brasileiro , encerrado ontem com o novo título palmeirense .
CF908-4: Diariamente , está *promovendo* desfiles 

# Pickle Bosque para posterior reconstituição

In [17]:
import pickle

for corpus in path:
    for sentence in list(root_without_subject[corpus].keys()):
        if not root_without_subject[corpus][sentence]:
            del root_without_subject[corpus][sentence]
        
    for sentence in list(subordinate_without_subject[corpus].keys()):
        if not subordinate_without_subject[corpus][sentence]:
            del subordinate_without_subject[corpus][sentence]

    if corpus == "bosque":
        with open("{}_root.p".format(corpus), "wb") as f:
            pickle.dump(root_without_subject[corpus], f)
        with open("{}_subordinate.p".format(corpus), "wb") as f:
            pickle.dump(subordinate_without_subject[corpus], f)

# Reconstituição

In [82]:
import estrutura_ud
import pickle

with open("bosque_root.p", "rb") as f:
    root = pickle.load(f)
with open("bosque_subordinate.p", "rb") as f:
    subordinate = pickle.load(f)
    
bosque = estrutura_ud.Corpus(recursivo=True)
bosque.load("bosque-ud-2.6.conllu")

dic_pronouns = {
    '1Sing': ['eu', 'eu'],
    '2Sing': ['tu', 'tu'],
    '3Sing': ['ele', 'ele'],
    '3SingFemale': ['ela', 'ele'],
    '1Plur': ['nós', 'nós'],
    '2Plur': ['vós', 'vós'],
    '3Plur': ['eles', 'ele'],
    '3PlurFemale': ['elas', 'ele']
}

bosque.sentences["CP493-7"].tokens[bosque.sentences["CP493-7"].map_token_id["26"]].deprel = "advcl"
bosque.sentences["CP493-7"].tokens[bosque.sentences["CP493-7"].map_token_id["26"]].dephead = "13"

In [83]:
def find_feats(feat, token):
    if not feat in token.feats:
        return ""
    return token.feats.split(feat + "=")[1].split("|")[0]

In [84]:
def insert_phrase(sentence, first_token_id, phrase):
    sintagmas_anteriores = []
    for t, token in enumerate(sentence.tokens):
        if t < sentence.map_token_id[first_token_id]:
            if token.dephead == first_token_id and (token.upos == "ADV" or (token.lemma == "se" and token.upos == "PRON") or token.deprel == "cop" or token.upos == "AUX"):
                is_sintagma = True
                for _token in sentence.tokens:
                    if _token.upos == "PUNCT" and _token.dephead == token.id:
                        is_sintagma = False
                        break
                if is_sintagma:
                    sintagmas_anteriores.append(sentence.tokens[t].id)
    who_head = first_token_id
    first_token_id = min(sintagmas_anteriores) if sintagmas_anteriores else first_token_id

    for new_token in reversed(phrase):
        token = estrutura_ud.Token()
        token.build(new_token.to_str())
        token.dephead = str(int(token.dephead) - int(token.id) + int(who_head.split("-")[0]))
        token.id = first_token_id.split("-")[0]
        if first_token_id == "1":
            if token.word != "SUJ_DESOCULTADO":
                token.word = token.word.title()
            sentence.tokens[0].word = sentence.tokens[0].word.lower()
        token.misc = "|".join(sorted([x for x in token.misc.split("|") + ["SUJ_DESOCULTADO"] if x != "_"]))
        sentence.tokens.insert(sentence.map_token_id[first_token_id], token)
        
        for t, token in enumerate(sentence.tokens):
            if t > sentence.map_token_id[first_token_id]:
                token.id = str(int(token.id)+1) if not '-' in token.id else str(int(token.id.split("-")[0])+1) + "-" + str(int(token.id.split("-")[1])+1)
            if token.dephead not in ["0", "_"] and int(token.dephead) >= int(first_token_id.split("-")[0]):
                token.dephead = str(int(token.dephead)+1)
        
        sentence.refresh_map_token_id()
                      
    sentence.metadados['text'] = " ".join([x.word for x in sentence.tokens if not '-' in x.id])
    return sentence

In [85]:
#ADVCL à esquerda com mesmo PESSNUM (root)
reconstituidos = []
for sent_id, tokens in list(root.items()):
    sentence = bosque.sentences[sent_id]
    sentence_qualifies = False
    to_add = {}
    for t in tokens:
        for _t, _token in enumerate(sentence.tokens):
            if _token.deprel == "advcl" and sentence.tokens[t].id == _token.dephead and _t < t and find_feats("Person", _token) == find_feats("Person", sentence.tokens[t]) and find_feats("Number", _token) == find_feats("Number", sentence.tokens[t]):
                for __token in sentence.tokens:
                    if __token.deprel in ["nsubj", "csubj", "nsubj:pass"] and __token.dephead == _token.id:
                        sentence_qualifies = True
                        to_add[t] = []
                        for ___token in sentence.tokens:
                            if ___token.dephead == __token.id or ___token.id == __token.id:
                                to_add[t].append(___token)
                        break
    
    if sentence_qualifies:
        for i, t in enumerate(to_add):
            bosque.sentences[sent_id] = insert_phrase(bosque.sentences[sent_id], str(int(bosque.sentences[sent_id].tokens[t+i].id)), to_add[t])
        del root[sent_id]
        reconstituidos.append(sent_id + ":\n" + bosque.sentences[sent_id].text + "\n" + bosque.sentences[sent_id].metadados['text'])

print("Reconstituídos: {} / {}".format(len(reconstituidos), len(root)+len(reconstituidos)))
print("\n" + "\n\n".join(reconstituidos))

Reconstituídos: 13 / 1439

CP40-3:
Quando o povo suíço recusou, em 92, a adesão ao Espaço Económico Europeu, como já fizera com a ONU, cometeu um grave engano.
Quando o povo suíço recusou , em 92 , a adesão a o Espaço Económico Europeu , como já fizera com a ONU , o povo suíço cometeu um grave engano .

CP88-5:
Só quando Moniz Pereira lhe surge na frente, se compenetra de que era mesmo verdade.
Só quando Moniz Pereira lhe surge em a frente , Moniz Pereira se compenetra de que era mesmo verdade .

CP94-5:
Se os chefes militares, actualmente, despacham com o ministro de 15 em 15 dias, farão o mesmo com o primeiro-ministro e o secretário de Estado governa o Ministério.
Se os chefes militares , actualmente , despacham com o ministro de 15 em 15 dias , os chefes militares farão o mesmo com o primeiro-ministro e o secretário de Estado governa o Ministério .

CP493-7:
Como o vírus tem uma propensão para infectar as células humanas, consegue em princípio fazer penetrar o gene dentro das célula

In [86]:
#ADVCL à esquerda com mesmo PESSNUM (subordinate)
reconstituidos = []
for sent_id, tokens in list(subordinate.items()):
    sentence = bosque.sentences[sent_id]
    sentence_qualifies = False
    to_add = {}
    for t in tokens:
        for _t, _token in enumerate(sentence.tokens):
            if _token.deprel == "advcl" and sentence.tokens[t].id == _token.dephead and _t < t and find_feats("Person", _token) == find_feats("Person", sentence.tokens[t]) and find_feats("Number", _token) == find_feats("Number", sentence.tokens[t]):
                for __token in sentence.tokens:
                    if __token.deprel in ["nsubj", "csubj", "nsubj:pass"] and __token.dephead == _token.id:
                        sentence_qualifies = True
                        to_add[t] = []
                        for ___token in sentence.tokens:
                            if ___token.dephead == __token.id or ___token.id == __token.id:
                                to_add[t].append(___token)
                        break
    
    if sentence_qualifies:
        for i, t in enumerate(to_add):
            bosque.sentences[sent_id] = insert_phrase(bosque.sentences[sent_id], str(int(bosque.sentences[sent_id].tokens[t+i].id)), to_add[t])
        del subordinate[sent_id]
        reconstituidos.append(sent_id + ":\n" + bosque.sentences[sent_id].text + "\n" + bosque.sentences[sent_id].metadados['text'])

print("Reconstituídos: {} / {}".format(len(reconstituidos), len(subordinate)+len(reconstituidos)))
print("\n" + "\n\n".join(reconstituidos))

Reconstituídos: 3 / 1937

CP636-2:
Os dirigentes da Fenprof avisam no entanto desde já que se Couto dos Santos insistir nalgumas das directrizes dos seus antecessores arranjará lenha para se queimar.
Os dirigentes de a Fenprof avisam em o entanto desde já que se Couto de os Santos insistir em algumas de as directrizes de os seus antecessores Couto Santos arranjará lenha para se queimar .

CF619-5:
Cabe perguntar: Se o Supremo não está de acordo com a suposta falta de regra das duas moedas, que existem de fato desde a emissão da última medida provisória, por que não a proclamou imediatamente inconstitucional e se serviu dela privadamente?
Cabe perguntar : Se o Supremo não está de acordo com a suposta falta de regra de as duas moedas , que existem de fato desde a emissão de a última medida provisória , por que o Supremo não a proclamou imediatamente inconstitucional e se serviu de ela privadamente ?

CF740-3:
Notamos ainda que, quando um passageiro brasileiro se inscreve num programa de 

In [87]:
#DESINÊNCIA (root)
reconstituidos = []
for sent_id, tokens in list(root.items()):
    sentence = bosque.sentences[sent_id]
    to_add = {}
    for t in tokens:
        person = find_feats("Person", sentence.tokens[t])
        number = find_feats("Number", sentence.tokens[t])
        gender = find_feats("Gender", sentence.tokens[t])
        feats = []
        if person:
            feats.append("Person=" + person)
        if number:
            feats.append("Number=" + number)
        if gender:
            feats.append("Gender=" + gender)
        feats = "|".join(sorted(feats)) if feats else "_"
        pronoun = person + number + gender
        if pronoun in dic_pronouns:
            pronoun = ["1", dic_pronouns[pronoun][0], dic_pronouns[pronoun][1], 'NOUN', '_', feats, '1', 'nsubj', '_', '_']
        else:
            pronoun = ["1", "SUJ_DESOCULTADO", "SUJ_DESOCULTADO", "NOUN", "_", feats, "1", "nsubj", "_", "_"]
        new_token = estrutura_ud.Token()
        new_token.build("\t".join(pronoun))
        to_add[t] = [new_token]
    
    for i, t in enumerate(to_add):
        bosque.sentences[sent_id] = insert_phrase(bosque.sentences[sent_id], str(int(bosque.sentences[sent_id].tokens[t+i].id)), to_add[t])
    del root[sent_id]
    reconstituidos.append(sent_id + ":\n" + bosque.sentences[sent_id].text + "\n" + bosque.sentences[sent_id].metadados['text'])

print("Reconstituídos: {} / {}".format(len(reconstituidos), len(root)+len(reconstituidos)))
print("\n" + "\n\n".join(reconstituidos))

Reconstituídos: 1426 / 1426

CF896-2:
Dos fumantes, 18,1 milhões são homens e 12,5 milhões, mulheres.
De os fumantes , 18,1 milhões SUJ_DESOCULTADO são homens e 12,5 milhões , mulheres .

CF899-2:
Na sua administração, executou um plano viário com largas avenidas interligando os bairros, que provocaram um boom imobiliário.
Em a sua administração , ele executou um plano viário com largas avenidas interligando os bairros , que provocaram um boom imobiliário .

CF902-3:
Quem não quiser beber durante o jantar, poderá levar o champanhe para casa.
Quem não quiser beber durante o jantar , ele poderá levar o champanhe para casa .

CF904-6:
É o que confirmam as estatísticas do Campeonato Brasileiro, encerrado ontem com o novo título palmeirense.
SUJ_DESOCULTADO é o que confirmam as estatísticas de o Campeonato Brasileiro , encerrado ontem com o novo título palmeirense .

CF908-4:
Diariamente, está promovendo desfiles de moda para seus consumidores.
Diariamente , SUJ_DESOCULTADO está promovendo 

In [88]:
#DESINÊNCIA (subordinate)
reconstituidos = []
for sent_id, tokens in list(subordinate.items()):
    sentence = bosque.sentences[sent_id]
    to_add = {}
    for t in tokens:
        person = find_feats("Person", sentence.tokens[t])
        number = find_feats("Number", sentence.tokens[t])
        gender = find_feats("Gender", sentence.tokens[t])
        feats = []
        if person:
            feats.append("Person=" + person)
        if number:
            feats.append("Number=" + number)
        if gender:
            feats.append("Gender=" + gender)
        feats = "|".join(sorted(feats)) if feats else "_"
        pronoun = person + number + gender
        if pronoun in dic_pronouns:
            pronoun = ["1", dic_pronouns[pronoun][0], dic_pronouns[pronoun][1], 'NOUN', '_', feats, '1', 'nsubj', '_', '_']
        else:
            pronoun = ["1", "SUJ_DESOCULTADO", "SUJ_DESOCULTADO", "NOUN", "_", feats, "1", "nsubj", "_", "_"]
        new_token = estrutura_ud.Token()
        new_token.build("\t".join(pronoun))
        to_add[t] = [new_token]
    
    for i, t in enumerate(to_add):
        bosque.sentences[sent_id] = insert_phrase(bosque.sentences[sent_id], bosque.sentences[sent_id].tokens[t+i].id, to_add[t])
    del subordinate[sent_id]
    reconstituidos.append(sent_id + ":\n" + bosque.sentences[sent_id].text + "\n" + bosque.sentences[sent_id].metadados['text'])

print("Reconstituídos: {} / {}".format(len(reconstituidos), len(subordinate)+len(reconstituidos)))
print("\n" + "\n\n".join(reconstituidos))

Reconstituídos: 1934 / 1934

CF893-4:
Para construir o ambiente futurista do filme, Godard recorreu quase que totalmente à fotografia.
Para SUJ_DESOCULTADO construir o ambiente futurista de o filme , Godard recorreu quase que totalmente a a fotografia .

CF895-2:
O problema é político porque envolve, por exemplo, a gratuidade da educação, da saúde, da previdência mínima.
O problema é político porque ele envolve , por exemplo , a gratuidade de a educação , de a saúde , de a previdência mínima .

CF895-3:
Excetuada a educação básica de primeiro grau, que deve ser gratuita e obrigatória, para todo o resto é preciso discriminar quem pode pagar.
SUJ_DESOCULTADO excetuada a educação básica de primeiro grau , que deve ser gratuita e obrigatória , para todo o resto é preciso discriminar quem pode pagar .

CF897-1:
PF prende dois por cobrar ágio
PF prende dois por SUJ_DESOCULTADO cobrar ágio

CF897-3:
A PF (Polícia Federal) indiciou ontem duas pessoas por vender carro com ágio.
A PF ( Polícia F

In [89]:
# Salvar modificado
bosque.save("bosque-ud-2.6-desocultado.conllu")

# Debugagem

In [ ]:
print(corpora['obras'].sentences["540"].to_str())

In [ ]:
#root_nominals["obras"]
root_without_subject["obras"]

In [29]:
print(subordinate)

{'CP344-5': [17, 26], 'CP345-1': [25], 'CP345-5': [0], 'CP345-9': [5], 'CP346-2': [3], 'CP348-1': [8], 'CP348-2': [14], 'CP348-4': [15], 'CP350-3': [11], 'CP350-5': [23], 'CP351-1': [23], 'CP352-1': [35], 'CP352-3': [30, 34, 44], 'CP352-5': [17], 'CP353-1': [23], 'CP353-4': [21, 24, 34], 'CP354-5': [9], 'CP355-5': [4, 9], 'CP355-7': [7], 'CP356-3': [0, 52], 'CP356-5': [22, 30], 'CP357-5': [9], 'CP357-8': [52, 63], 'CP357-7': [49, 67], 'CP358-4': [45], 'CP358-7': [10, 70], 'CP359-3': [20], 'CP359-4': [29], 'CP361-3': [38], 'CP365-2': [19], 'CP366-1': [57], 'CP366-6': [22, 33], 'CP367-2': [53, 57], 'CP368-4': [21], 'CP368-6': [45], 'CP373-3': [1], 'CP373-4': [35], 'CP376-1': [16, 21], 'CP376-3': [27, 68], 'CP380-1': [7], 'CP381-1': [11], 'CP381-2': [21], 'CP381-5': [8], 'CP381-7': [8], 'CP381-8': [27], 'CP382-2': [1, 18], 'CP382-4': [14], 'CP383-1': [4], 'CP383-2': [4, 29], 'CP384-11': [14, 26], 'CP385-4': [14, 33], 'CP387-2': [19], 'CP388-1': [14], 'CP390-1': [3], 'CP390-3': [2], 'CP390

In [61]:
bosque.sentences["CP493-7"].tokens[bosque.sentences["CP493-7"].map_token_id["29"]].deprel = "advcl"
bosque.sentences["CP493-7"].tokens[bosque.sentences["CP493-7"].map_token_id["29"]].dephead = "16"

# text = Como o vírus tem uma propensão para infectar as células humanas , o vírus servindo consegue em princípio fazer penetrar o gene dentro de as células cancerosas , servindo de « cavalo de Tróia » a os genes p53 .
# sent_id = CP493-7
# source = CETEMPúblico n=493 sec=clt-soc sem=95a
# id = 2590
1	Como	como	SCONJ	_	_	4	mark	_	_
2	o	o	DET	_	Definite=Def|Gender=Masc|Number=Sing|PronType=Art	3	det	_	_
3	vírus	vírus	NOUN	_	Gender=Masc|Number=Sing	4	nsubj	_	_
4	tem	ter	VERB	_	Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin	16	advcl	_	_
5	uma	um	DET	_	Definite=Ind|Gender=Fem|Number=Sing|PronType=Art	6	det	_	_
6	propensão	propensão	NOUN	_	Gender=Fem|Number=Sing	4	obj	_	_
7	para	para	SCONJ	_	_	8	mark	_	_
8	infectar	infectar	VERB	_	VerbForm=Inf	6	acl	_	_
9	as	o	DET	_	Definite=Def|Gender=Fem|Number=Plur|PronType=Art	10	det	_	_
10	células	célula	NOUN	_	Gender=Fem|Number=Plur	8	obj	_	_
11	humanas	humano	ADJ	_	Gender=Fem|Number=Plur	10	amod	_	SpaceAfter=No
12	,	,	PUNCT	_	_	4	punct	_	_
13	